In [9]:
import cv2
import mediapipe as mp
import math, os, json
import numpy as np
from datetime import datetime
from pathlib import Path

# ---------- optional PDF ----------
try:
    from fpdf import FPDF
    _PDF_OK = True
except Exception:
    _PDF_OK = False

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

finger_joints = {
    "thumb": [1, 2, 3, 4],
    "index": [5, 6, 7, 8],
    "middle": [9, 10, 11, 12],
    "ring": [13, 14, 15, 16],
    "pinky": [17, 18, 19, 20]
}

def diameter_to_us_ring_size(mm):
    table = {
        14.0: 3, 14.4: 3.5, 14.8: 4, 15.2: 4.5, 15.6: 5,
        16.0: 5.5, 16.5: 6, 16.9: 6.5, 17.3: 7, 17.7: 7.5,
        18.1: 8, 18.5: 8.5, 19.0: 9, 19.4: 9.5, 19.8: 10,
        20.2: 10.5, 20.6: 11, 21.0: 11.5, 21.4: 12, 21.8: 12.5, 22.2: 13
    }
    closest = min(table.keys(), key=lambda x: abs(x - mm))
    return table[closest]

def ensure_dir(p): Path(p).mkdir(parents=True, exist_ok=True)
def stamp(): return datetime.now().strftime("%Y%m%d_%H%M%S")

def euclidean_distance(p1, p2, w, h):
    x1, y1 = int(p1.x * w), int(p1.y * h)
    x2, y2 = int(p2.x * w), int(p2.y * h)
    return math.hypot(x2 - x1, y2 - y1), (x1, y1), (x2, y2)

def save_json(output_dir, base, measurements, meta):
    ensure_dir(output_dir)
    path = Path(output_dir) / f"{base}.json"
    payload = {
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "scale_mm_per_pixel": meta.get("scale_mm_per_pixel"),
        "reference": {
            "reference_mm": meta.get("reference_mm"),
            "reference_pixels": meta.get("reference_pixels"),
        },
        "image_path": meta.get("image_path"),
        "notes": meta.get("notes", []),
        "fingers": measurements,
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    return str(path)

def save_pdf(output_dir, base, image_path, measurements, meta):
    if not _PDF_OK:
        print("[WARN] fpdf2 not installed. Run: pip install fpdf2")
        return None
    ensure_dir(output_dir)
    pdf_path = Path(output_dir) / f"{base}.pdf"
    pdf = FPDF(unit="mm", format="A4"); pdf.add_page(); pdf.set_auto_page_break(True, 15)
    pdf.set_font("Helvetica", "B", 16); pdf.cell(0, 10, "Hand & Ring Size Report", ln=True)
    pdf.set_font("Helvetica", "", 11)
    pdf.cell(0, 6, f"Timestamp: {datetime.now().isoformat(timespec='seconds')}", ln=True)
    pdf.cell(0, 6, f"Scale (mm/px): {meta.get('scale_mm_per_pixel') or 'N/A'}", ln=True)
    ref_mm, ref_px = meta.get("reference_mm"), meta.get("reference_pixels")
    pdf.cell(0, 6, f"Reference: {ref_mm} mm over {ref_px} px" if (ref_mm and ref_px) else "Reference: None", ln=True)
    pdf.ln(4); pdf.set_font("Helvetica", "B", 12); pdf.cell(0, 8, "Annotated Image:", ln=True)
    if image_path and Path(image_path).exists():
        pdf.image(str(image_path), w=180)
    else:
        pdf.set_font("Helvetica", "I", 11); pdf.cell(0, 8, "(No image)", ln=True)
    pdf.ln(6); pdf.set_font("Helvetica", "B", 12)
    headers = ["Finger", "Span (px)", "Diameter (mm)", "US Size"]; col_w = [40, 40, 40, 40]
    for i, h in enumerate(headers): pdf.cell(col_w[i], 8, h, border=1, align="C")
    pdf.ln(8); pdf.set_font("Helvetica", "", 11)
    for finger, vals in measurements.items():
        px = vals.get("pixels"); mm = vals.get("diameter_mm"); us = vals.get("us_ring_size")
        pdf.cell(col_w[0], 8, finger.capitalize(), border=1)
        pdf.cell(col_w[1], 8, f"{px:.1f}" if isinstance(px, (int,float)) else "—", border=1, align="C")
        pdf.cell(col_w[2], 8, f"{mm:.2f}" if isinstance(mm, (int,float)) else "—", border=1, align="C")
        pdf.cell(col_w[3], 8, str(us) if us is not None else "N/A", border=1, align="C")
        pdf.ln(8)
    pdf.output(str(pdf_path)); return str(pdf_path)

def write_report_files(output_dir, annotated_image, measurements, meta):
    ensure_dir(output_dir); base = f"ring_report_{stamp()}"
    img_path = None
    if annotated_image is not None:
        img_path = Path(output_dir) / f"{base}.png"
        cv2.imwrite(str(img_path), annotated_image)
        meta = {**meta, "image_path": str(img_path)}
    else:
        meta = {**meta, "image_path": None}
    json_path = save_json(output_dir, base, measurements, meta)
    pdf_path = save_pdf(output_dir, base, meta.get("image_path"), measurements, meta)
    print("[INFO] Saved:"); print("  JSON:", json_path)
    if pdf_path: print("  PDF :", pdf_path)
    if img_path: print("  IMG :", img_path)
    return {"json": json_path, "pdf": pdf_path, "image": str(img_path) if img_path else None}

def measure_fingers(image, results, scale_mm_per_px=None):
    h, w, _ = image.shape
    measurements = {}
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1)
        )
        for finger, ids in finger_joints.items():
            left_point  = hand_landmarks.landmark[ids[1]]
            right_point = hand_landmarks.landmark[ids[2]]
            dist_px, p1, p2 = euclidean_distance(left_point, right_point, w, h)
            dist_mm = dist_px * scale_mm_per_px if scale_mm_per_px else None
            ring_size = diameter_to_us_ring_size(dist_mm) if isinstance(dist_mm,(int,float,np.floating)) else None
            measurements[finger] = {"pixels": float(dist_px),
                                    "diameter_mm": float(dist_mm) if dist_mm is not None else None,
                                    "us_ring_size": ring_size}
            cv2.line(image, p1, p2, (0,255,0), 1)
            y0 = max(12, p1[1]-10)
            cv2.putText(image, f"{finger}: {dist_px:.1f}px", (p1[0], y0),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)
            if dist_mm is not None:
                cv2.putText(image, f"{dist_mm:.1f}mm  US:{ring_size}", (p1[0], y0-15),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,0), 1)
            else:
                cv2.putText(image, "Set scale for US size", (p1[0], y0-15),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255,0,0), 1)
    return image, measurements

def try_open_camera(preferred_index=None, max_try=4):
    # Prefer an index if given; otherwise scan 0..3
    indices = [preferred_index] if preferred_index is not None else list(range(max_try))
    for idx in indices:
        cap = cv2.VideoCapture(idx)
        if cap.isOpened():
            print(f"[INFO] Camera opened at index {idx}")
            return cap
        cap.release()
    print("[ERR] Could not open any camera (tried indexes 0..3).")
    return None

def video_mode(reference_mm=None, reference_pixels=None, output_dir="reports", camera_index=None):
    scale = (reference_mm / reference_pixels) if (reference_mm and reference_pixels) else None
    cap = try_open_camera(camera_index)
    if cap is None:
        print(" - Close other apps using the camera (Zoom/Teams/etc.)")
        print(" - On Windows, allow camera for Python in Privacy settings.")
        print(" - If you’re in WSL or a headless server, GUI/camera may not be available.")
        return

    print("Press [S] to save PNG+JSON+PDF, [Q] to quit.")
    last_frame, last_meas = None, None
    with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
        while True:
            ok, frame = cap.read()
            if not ok:
                print("[ERR] Camera frame not available."); break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb)
            annotated = frame.copy()
            if results.multi_hand_landmarks:
                annotated, meas = measure_fingers(annotated, results, scale)
                last_frame, last_meas = annotated, meas
            cv2.rectangle(annotated, (10,10), (460,35), (0,0,0), -1)
            cv2.putText(annotated, "Q=Quit  S=Save report  (set scale for US sizes)",
                        (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1)
            cv2.imshow("Finger Measurement", annotated)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            if key == ord('s'):
                if last_frame is None or last_meas is None:
                    print("[INFO] Nothing to save yet.")
                else:
                    meta = {
                        "scale_mm_per_pixel": scale,
                        "reference_mm": reference_mm,
                        "reference_pixels": reference_pixels,
                        "notes": ["Video mode snapshot export"]
                    }
                    write_report_files(output_dir, last_frame, last_meas, meta)
    cap.release(); cv2.destroyAllWindows()

if __name__ == "__main__":
    # If you have a known scale, set reference_mm & reference_pixels here (e.g., a coin in frame)
    # video_mode(reference_mm=24, reference_pixels=50, output_dir="reports")
    video_mode(output_dir="reports")  # default run


[INFO] Camera opened at index 0
Press [S] to save PNG+JSON+PDF, [Q] to quit.


C:\ProgramData\anaconda3\envs\handsize2\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[WARN] fpdf2 not installed. Run: pip install fpdf2
[INFO] Saved:
  JSON: reports\ring_report_20250924_045606.json
  IMG : reports\ring_report_20250924_045606.png
[WARN] fpdf2 not installed. Run: pip install fpdf2
[INFO] Saved:
  JSON: reports\ring_report_20250924_045612.json
  IMG : reports\ring_report_20250924_045612.png
[WARN] fpdf2 not installed. Run: pip install fpdf2
[INFO] Saved:
  JSON: reports\ring_report_20250924_045617.json
  IMG : reports\ring_report_20250924_045617.png
